<a href="https://colab.research.google.com/github/dream80/DeepFaceLab_Colab/blob/master/DeepFaceLab_Colab_zh_cn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 简介

无需多说，一路运行即可！

此脚本由**托尼是塔克**创建，仅用于学习，请勿商用，请勿用于非法用途，否者后果自负。

# 第一步 准备好workspace
这一部你可以选着两种方式
1. 使用默认的workspace，你无需自己上传，仅用于熟悉操作。
2. 通过Google Drive （谷歌云盘）上传自己的workspace到指定目录。


注意：谷歌网址现在都需要“科学上网”才能访问。

In [0]:
#挂载谷歌云盘
#点击链接授权，复制授权码，填入框框，然后回车。

from google.colab import drive

drive.mount('/content/drive', force_remount=True)

In [0]:
#创建DeepFaceLab目录，并且进入目录
%cd /content/drive/My Drive/
!mkdir DeepFaceLab
%cd /content/drive/My Drive/DeepFaceLab/

In [0]:
# 下面的1和2 不用同时点，二选一！！！！
# 1. 练手可以使用这一行diam，直接git clone一个workspace
!git clone https://github.com/dream80/DFLWorkspace.git workspace

In [0]:
# 2.正常情况肯定是通过谷歌云盘上传自己的workspace，上传前压缩一下，然后用下面的命令解压。
# 注意一定要上传到DeepFaceLab目录下。
!unzip workspace.zip



#第二步 安装DeepFaceLab


In [0]:
# 获取DFL源代码v1.1稳定版

!git clone -b v1.1 https://github.com/dream80/DeepFaceLab_Colab.git

# 获取最新代代码，用这条命令，有风险。
#!git clone https://github.com/dream80/DeepFaceLab_Colab.git

In [0]:
# 进入DeepFaceLab_Colab目录
%cd /content/drive/My Drive/DeepFaceLab/DeepFaceLab_Colab

#获取最新源代码
#!git pull


In [0]:
# 安装Python依赖
!pip install -r requirements_colab.txt
!pip install --upgrade scikit-image

# 第三步. 提取脸部

In [0]:
# 确保路径正确，进入DeepFaceLab_Colab

%cd /content/drive/My Drive/DeepFaceLab/DeepFaceLab_Colab

In [0]:
#Src 视频转图片

!python main.py videoed extract-video --input-file ../workspace/data_src.mp4 --output-dir ../workspace/data_src/

In [0]:
#Src 提取脸部图片

!python main.py extract --input-dir ../workspace/data_src --output-dir ../workspace/data_src/aligned --detector s3fd --debug-dir ../workspace/data_src/aligned_debug

In [0]:
#Src排序，可以通过谷歌云盘查看结果，删除不良图片

!python main.py sort --input-dir ../workspace/data_src/aligned --by hist

In [0]:
#Dst视频转图片

!python main.py videoed extract-video --input-file ../workspace/data_dst.mp4 --output-dir ../workspace/data_dst/

In [0]:
#Dst提取脸部图片

!python main.py extract --input-dir ../workspace/data_dst --output-dir ../workspace/data_dst/aligned --detector s3fd --debug-dir ../workspace/data_dst/aligned_debug

In [0]:
#Dst排序，可以通过谷歌云盘查看结果，删除不良图片

!python main.py sort --input-dir ../workspace/data_dst/aligned --by hist

# 第四步. 训练模型


*  支持H128,SAE,DF, LIAEF128等模型，根据自己的情况选择模型。
*  训练开始是需要配置参数，记得开启预览，其他参数根据自己情况选择，使用默认参数的话直接回车即可。
*  不想训练了可以点击停止，停止时会抛出异常，但是没什么关系。下次可以继续训练
*  如果想要查看history ，先停止训练，然后点击下面第三段代码

In [0]:
# 确保路径正确，进入DeepFaceLab_Colab

%cd /content/drive/My Drive/DeepFaceLab/DeepFaceLab_Colab

**下面是启动模型训练的脚本，不要全部点，只要点其中一个。默认使用第一个SAE**

In [0]:
# Running trainer. SAE 

!python main.py train --training-data-src-dir ../workspace/data_src/aligned --training-data-dst-dir ../workspace/data_dst/aligned --model-dir ../workspace/model --model SAE --no-preview

In [0]:
# Running trainer  H128

!python main.py train --training-data-src-dir ../workspace/data_src/aligned --training-data-dst-dir ../workspace/data_dst/aligned --model-dir ../workspace/model --model H128 --no-preview

In [0]:
# Running trainer. DF 

!python main.py train --training-data-src-dir ../workspace/data_src/aligned --training-data-dst-dir ../workspace/data_dst/aligned --model-dir ../workspace/model --model DF --no-preview

In [0]:
# Running trainer. LIAEF128 

!python main.py train --training-data-src-dir ../workspace/data_src/aligned --training-data-dst-dir ../workspace/data_dst/aligned --model-dir ../workspace/model --model LIAEF128 --no-preview

**模型预览请使用独立的colab脚本：**
https://github.com/dream80/DeepFaceLab_Colab/blob/master/ViewLastHistory.ipynb


# 第五步. 转换输出

默认使用SAE进行转换，如果需要其他模型，请修改命令中的参数。

比如将 --model SAE 修改为 --model H128

In [0]:
#用src中的脸替换dst的脸 
!python main.py convert --input-dir ../workspace/data_dst --output-dir ../workspace/data_dst/merged --aligned-dir ../workspace/data_dst/aligned --model-dir ../workspace/model --model SAE

In [0]:
#把替换好的图片转换成视频

!python main.py videoed video-from-sequence --input-dir ../workspace/data_dst/merged --output-file ../workspace/result.mp4 --reference-file ../workspace/data_dst.mp4

#第六步. 继续训练
当你第二次开始训练，或者掉线之后继续训练时并不需要执行上面所有的步骤。只需要下面简单的几个步骤。
1. 挂载云盘
2. 安装依赖
3. 开始训练

In [0]:
#挂载谷歌云盘
#点击链接授权，复制授权码，填入框框，然后回车。

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# 进入DeepFaceLab_Colab目录
%cd /content/drive/My Drive/DeepFaceLab/DeepFaceLab_Colab

# 安装Python依赖
!pip install -r requirements_colab.txt
!pip install --upgrade scikit-image

# 开始训练SAE ，如果是其他模型，修改后面的参数即可。

!python main.py train --training-data-src-dir ../workspace/data_src/aligned --training-data-dst-dir ../workspace/data_dst/aligned --model-dir ../workspace/model --model SAE --no-preview

# 其他

 谷歌云地址：https://drive.google.com/drive/my-drive

 作者邮箱 ：wpgdream@gmail.com
 
 使用教程：http://deepfakescn.com
 
 工具脚本：https://github.com/dream80/DeepFaceLab_Colab/blob/master/Tools.ipynb
 
 
不求别的，只求在github给个小星星^_^

https://github.com/dream80/DeepFaceLab_Colab

右上角star 谢谢!!!